In [1]:
import cv2
import sys
import os
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(str(parent_dir))

tif = cv2.imread("example_images/images/original.tif", cv2.IMREAD_UNCHANGED)
cv2.imwrite("example_original.png", tif)


True

In [1]:
import cv2
import numpy as np
import logging

class imgProcessor:
    def __init__(self, image_path: str, sr_map_path: None):
        self.logger = logging.getLogger(self.__class__.__name__)

        # Get image and map
        self.image_path = image_path
        self.image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        if self.image is None:
            raise ValueError(f"Failed to load image from {image_path}")
        self.sr_map = cv2.imread(sr_map_path, cv2.IMREAD_UNCHANGED)
        if self.sr_map is None:
            raise ValueError(f"Failed to load map from {sr_map_path}")

        # Attributes
        self.roi = None  # (x, y, w, h)
        self.roi_image = None

        self.logger.info(f"Initialized {self.__class__.__name__}")
        self.logger.info(f"Image loaded: {self.image.shape}")
        self.logger.info(f"Map loaded: {self.sr_map.shape}")

    
    def convert_img_to_log_space(self, linear_img) -> np.ndarray:
        """
        Converts a 16-bit linear image to log space, setting linear 0 values to 0 in log space.

        Parameters:
        -----------
        linear_img : np.array
            Input 16-bit image as a NumPy array.

        Returns:
        --------
        log_img : np.array
            Log-transformed image with 0 values preserved.
        """

        log_img = np.zeros_like(linear_img, dtype = np.float32)
        log_img[linear_img != 0] = np.log(linear_img[linear_img != 0])
        assert np.min(log_img) >= 0 and np.max(log_img) <= 11.1

        return log_img

    def log_to_linear(self, log_img) -> None:
        """
        Converts log transofrmed image back to linear space in 16 bits.

        Parameters:
        -----------
        log_img : np.array
            Log-transformed image with values between 0 and 11.1.

        Returns:
        --------
        linear_img : np.array
            Visualization-ready 8-bit image.
        """

        linear_img = np.exp(self.img_chroma)
        return linear_img
    
    def convert_16bit_to_8bit(self, img) -> None:
        """
        Converts a 16-bit image to 8-bit by normalizing pixel values.

        Parameters:
        -----------
        img : np.array
            Input image array in 16-bit format (dtype: np.uint16).
        
        Returns:
        --------
        img_8bit : np.array
            Output image array converted to 8-bit (dtype: np.uint8).
        """
        img_normalized = np.clip(img / 255, 0, 255)
        img_8bit = np.uint8(img_normalized)
        return img_8bit

    def show_image(self, use_roi=False):
        display_img = self.roi_image if (use_roi and self.roi_image is not None) else self.image
        window_name = "ROI Image" if use_roi else "Full Image"
        cv2.imshow(window_name, display_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    def select_roi(self):
        self.logger.info("[srLight] Select ROI using mouse...")
        roi = cv2.selectROI("Select ROI", self.image, showCrosshair=True, fromCenter=False)
        cv2.destroyWindow("Select ROI")

        if roi == (0, 0, 0, 0):
            self.logger.info("[srLight] No ROI selected.")
            return

        self.roi = roi
        x, y, w, h = roi
        self.roi_image = self.image[y:y+h, x:x+w]
        self.logger.info(f"[srLight] ROI selected: x={x}, y={y}, w={w}, h={h}")
    
    def shift_pixels_along_isd(image_log: np.ndarray, isd_map: np.ndarray, roi: tuple, distance: float) -> np.ndarray:
        """
        Shift pixels in log-RGB space along their ISD vector by a given distance within the ROI.
        
        Args:
            image_log (np.ndarray): Log-RGB image of shape (H, W, 3)
            isd_map (np.ndarray): ISD map of shape (H, W, 3), should be unit vectors
            roi (tuple): (x, y, w, h) rectangle specifying the ROI
            distance (float): Distance to shift pixels along ISD vector

        Returns:
            np.ndarray: Modified log-RGB image with ROI pixels shifted
        """
        shifted_image = image_log.copy()
        x, y, w, h = roi

        roi_pixels = image_log[y:y+h, x:x+w]
        roi_isd = isd_map[y:y+h, x:x+w]

        # Shift each pixel in the ROI
        shifted_roi = roi_pixels + distance * roi_isd

        # Replace back into image
        shifted_image[y:y+h, x:x+w] = shifted_roi
        return shifted_image

In [9]:
import tifffile
import cv2
import numpy as np

# Load the original float32 ISD map
sr_map_path = "/Users/mikey/Library/Mobile Documents/com~apple~CloudDocs/Code/code_projects/isd-annotator/annotations/images_for_testing/folder_6/isd_maps/maddineni_poojit_045_isd.tiff"

isd_float = tifffile.imread(sr_map_path)
print("Original:", isd_float.dtype, isd_float.min(), isd_float.max())

# Load with OpenCV (should fail silently or convert to uint16)
isd_cv = cv2.imread(sr_map_path, cv2.IMREAD_UNCHANGED)
isd_cv = cv2.cvtColor(isd_cv, cv2.COLOR_BGR2RGB)

print("OpenCV:", isd_cv.dtype, isd_cv.min(), isd_cv.max())


diff = np.abs(isd_cv - isd_float).mean()
print("Mean absolute difference (uint16 range scaled):", diff)

Original: float32 28359.771 44193.09
OpenCV: float32 28359.771 44193.09
Mean absolute difference (uint16 range scaled): 0.0


In [10]:
import torch
print(torch.backends.mps.is_available())     # True if MPS is available
print(torch.backends.mps.is_built())         # True if PyTorch was built with MPS

True
True
